# Advanced features

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas package

Make sure to install it using `pip install bipartitepandas`.

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

For this notebook, we simulate data.

In [3]:
df = bpd.SimBipartite().simulate()
bdf = bpd.BipartiteDataFrame(i=df['i'], j=df['j'], y=df['y'], t=df['t'])
display(bdf)

,i,j,y,t
0,0,122,2.014468,0
1,0,122,-0.603558,1
2,0,122,2.489361,2
3,0,71,0.180421,3
4,0,71,-0.625300,4
...,...,...,...,...
49995,9999,126,-0.085808,0
49996,9999,139,0.779111,1
49997,9999,139,1.162888,2
49998,9999,139,-0.072980,3


## Advanced data cleaning

<span class="label label-success">Hint</span> Want details on all cleaning parameters? Run `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then run `bpd.clean_params().describe(key)`.

#### Compute the largest connected component

Use the parameter `connectedness` to set the connectedness measure to use, and `component_size_variable` to choose how to measure component size (e.g. `firms` chooses the connected component with the greatest number of unique firm ids, while `workers` chooses the connected component with the greatest number of unique worker ids).

<span class="label label-info">Note</span> Connectedness is NOT NECESSARILY maintained between non-collapsed and collapsed formats. Therefore, if you plan to use connected, collapsed data, it is recommended to first clean the non-collapsed data with `connectedness=None`, next collapse the data, and finally clean the collapsed data with the connectedness measure you plan to use.

#### Set how to handle worker-year duplicates

Use the parameter `i_t_how` to customize how worker-year duplicates are handled.

#### Collapse at the match-level

If you drop the `t` column, collapsing will automatically collapse at the match level. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Avoid unnecessary copies

If you are working with a large dataset, you will want to avoid copies whenever possible. So set `copy=False`.

#### Avoid unnecessary sorts

If you know your data is sorted by `i` and `t` (or, if you aren't including a `t` column, just by `i`), then set `is_sorted=True`.

#### Avoid complicated loops

Sometimes workers leave a firm, then return to it (we call these workers *returners*). Returners can cause computational difficulties because sometimes intermediate firms get dropped (e.g. a worker goes from firm $A \to B \to A$, but firm $B$ gets dropped). This turns returners into stayers. This can change the largest connected set of firms, and if data is in collapsed format, requires the data to be recollapsed.

Because of these potential complications, if there are returners, many methods require loops that run until convergence.

These difficulties can be avoided by setting the parameter `drop_returns` (there are multiple ways to handle returners, they can be seen by running `bpd.clean_params().describe('drop_returns')`).

<span class="label label-default">Alternative</span> Another way to handle returners is to drop the `t` column. Then, sorting will automatically sort by `i` and `j`, which eliminates the possibility of returners. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

## Advanced clustering

#### Install Intel(R) Extension for Scikit-learn

Intel(R) Extension for Scikit-learn ([GitHub](https://github.com/intel/scikit-learn-intelex)) can speed up KMeans clustering.

## Advanced dataframe handling

#### Disable logging

Logging can slow down basic operations on BipartitePandas dataframes (e.g. data cleaning). Set the parameter `log=False` when constructing your dataframe to turn off logging.

#### Use method chaining with in-place operations

Unlike standard Pandas, BipartitePandas allows method chaining with in-place operations.

#### Understand the distinction between general columns and subcolumns

Users interact with general columns, while BipartitePandas dataframes display subcolumns. As an example, for event study format, the columns for firm clusters are labeled `g1` and `g2`. These are the subcolumns for general column `g`. If you want to drop firm clusters from the dataframe, rather than dropping `g1` and `g2` separately, you must drop the general column `g`. This paradigm applies throughout BipartitePandas and the documentation will make clear when you should specify general columns.

#### Simpler constructor

If the columns in your Pandas dataframe are already named correctly, you can simply put the dataframe as a parameter into the BipartitePandas dataframe constructor. Here is an example:

In [4]:
bdf = bpd.BipartiteDataFrame(df).clean()
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,122,2.014468,0,0,0.000000,6,2,0.348756
1,0,122,-0.603558,1,0,0.000000,6,2,0.348756
2,0,122,2.489361,2,1,0.000000,6,2,0.348756
3,0,71,0.180421,3,1,0.000000,3,2,-0.348756
4,0,71,-0.625300,4,0,0.000000,3,2,-0.348756
...,...,...,...,...,...,...,...,...,...
49995,9999,126,-0.085808,0,1,0.430727,6,3,0.348756
49996,9999,139,0.779111,1,1,0.430727,7,3,0.604585
49997,9999,139,1.162888,2,0,0.430727,7,3,0.604585
49998,9999,139,-0.072980,3,0,0.430727,7,3,0.604585


## Restoring original ids

To restore original ids, we need to make sure the dataframe is tracking ids as they change.

We make sure the dataframe tracks ids as they change by setting `include_id_reference_dict=True`.

Notice that in this example we use `j / 2`, so that `j` will be modified during data cleaning.

The method `.original_ids()` will then return a dataframe that merges in the original ids.

In [5]:
bdf_original_ids = bpd.BipartiteDataFrame(i=df['i'], j=(df['j'] / 2), y=df['y'], t=df['t'], include_id_reference_dict=True).clean()
display(bdf_original_ids.original_ids())

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,original_j
0,0,0,2.014468,0,0,61.0
1,0,0,-0.603558,1,0,61.0
2,0,0,2.489361,2,1,61.0
3,0,1,0.180421,3,1,35.5
4,0,1,-0.625300,4,0,35.5
...,...,...,...,...,...,...
49995,9999,111,-0.085808,0,1,63.0
49996,9999,181,0.779111,1,1,69.5
49997,9999,181,1.162888,2,0,69.5
49998,9999,181,-0.072980,3,0,69.5


## Comparing dataframes

Dataframes can be compared using the utility method `bpd.util.compare_frames()`.

In [6]:
bpd.util.compare_frames(bdf, bdf.iloc[:len(bdf) // 2], size_variable='len', operator='geq')

True

## Filling in missing periods as unemployed

The method `.fill_missing_periods()` (for *Long* format) will fill in rows for missing intermediate periods. Note that this method will not work with custom columns.

<span class="label label-success">Hint</span> Filling in missing periods is a useful way to make sure that `.collapse()` only collapses over worker-firm spells if they are for contiguous periods.

In this example, we drop periods 1-3, then fill them in, setting `alpha` to become -1:

In [7]:
bdf_missing = bdf[(bdf['t'] == 0) | (bdf['t'] == 4)].clean()
display(bdf_missing.fill_missing_periods({'alpha': -1}))

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,122,2.014468,0,1,0.000000,6,2,0.348756
1,0,-1,NaN,1,<NA>,-1.000000,<NA>,<NA>,NaN
2,0,-1,NaN,2,<NA>,-1.000000,<NA>,<NA>,NaN
3,0,-1,NaN,3,<NA>,-1.000000,<NA>,<NA>,NaN
4,0,71,-0.625300,4,1,0.000000,3,2,-0.348756
...,...,...,...,...,...,...,...,...,...
49995,9999,126,-0.085808,0,1,0.430727,6,3,0.348756
49996,9999,-1,NaN,1,<NA>,-1.000000,<NA>,<NA>,NaN
49997,9999,-1,NaN,2,<NA>,-1.000000,<NA>,<NA>,NaN
49998,9999,-1,NaN,3,<NA>,-1.000000,<NA>,<NA>,NaN


## Getting extended event study dataframes

BipartitePandas allows you to use *Long* format data to generate event studies with more than 2 periods.

You can specify:

- which column signals a transition (e.g. if `j` is used, a transition is when a worker moves firms)
- which column(s) should be treated as the event study outcome
- how many periods before and after the transition should be considered
- whether the pre- and/or post-trends must be stable, and for which column(s)

We consider an example where `j` is the transition column, `y` is the outcome column, and with pre- and post-trends of length 2 that are required to be at the same firm. Note that `y_f1` is the first observation after the individual moves firms.

In [8]:
es_extended = bdf.get_extended_eventstudy(transition_col='j', outcomes='y', periods_pre=2, periods_post=2, stable_pre='j', stable_post='j')
display(es_extended)

,i,t,y_l2,y_l1,y_f1,y_f2
0,0,3,-0.603558,2.489361,0.180421,-0.625300
1,1,2,-1.668656,0.242102,0.503344,0.289625
2,13,3,-0.110084,-0.083841,0.503936,-1.819574
3,15,3,-1.154242,-1.417423,-2.611288,-2.143656
4,18,2,0.369764,1.556126,-1.160716,0.324391
...,...,...,...,...,...,...
2536,9990,3,-0.504179,0.339318,-1.628170,-2.681138
2537,9991,2,-0.322445,0.096851,-1.707640,-1.620544
2538,9992,3,1.636948,-0.500021,1.961618,-1.063378
2539,9995,2,1.429262,0.491578,2.449732,1.612246


## Advanced data simulation

For details on all simulation parameters, run `bpd.sim_params().describe_all()`, or search through `bpd.sim_params().keys()` for a particular key, and then run `bpd.sim_params().describe(key)`.